# Preprocessing data for train set

## to_ann_39 & to_transformer

In [ ]:
import pandas as pd
import ast
import numpy as np

# Đọc CSV
df = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/test_not_aug.csv')

# Hàm chuyển predicted_vector từ string thành list
def to_ann_39(vec_str):
    return np.array(ast.literal_eval(vec_str)).flatten()

def to_transformer(vec_str):
    return np.array(ast.literal_eval(vec_str))

# Áp dụng hàm cho từng bản ghi để chuyển đổi predicted_vector từ string thành numpy array
df['to_ann_39'] = df['predicted_vector'].apply(to_ann_39)
df['to_transformer'] = df['predicted_vector'].apply(to_transformer)

print(type(df['to_ann_39'][0]))      # numpy.ndarray
print(df['to_ann_39'][0].shape)      # (39,)

print(type(df['to_transformer'][0]))   # numpy.ndarray
print(df['to_transformer'][0].shape)   # (13, 3)

In [ ]:
print(df['to_ann_39'][0])
print(df['to_transformer'][0])

## reduce dimension

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Standardize dữ liệu
X = np.vstack(df['to_ann_39'].values)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sử dụng PCA
pca = PCA()
pca.fit(X_scaled)

# Tính variance explained ratio
explained_variance = pca.explained_variance_ratio_

# Elbow Method
plt.plot(range(1, len(explained_variance)+1), np.cumsum(explained_variance), marker='o', linestyle='--')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Elbow Method')
plt.show()

In [ ]:
import pickle

# Sử dụng PCA giữ lại 95% thông tin
pca_95 = PCA(0.95)
X_pca_95 = pca_95.fit_transform(X_scaled)

# In ra số chiều sau khi giảm
print(f'Số chiều sau khi giảm: {X_pca_95.shape[1]}')

# Lưu kết quả PCA vào cột mới trong DataFrame
df['pca95_to_ann'] = list(X_pca_95)

# Lưu cấu trúc PCA để áp dụng cho tập val và test
with open('pca_95.pkl', 'wb') as f:
    pickle.dump(pca_95, f)

In [ ]:
# Chọn số chiều phù hợp từ Elbow Method
n_components_optimal = np.argmax(np.cumsum(explained_variance) >= 0.90) + 1  # Ví dụ chọn số chiều khi explained variance đạt 90%

print(f'Số chiều tối ưu: {n_components_optimal}')

# Sử dụng PCA với số chiều từ Elbow Method
pca_elbow = PCA(n_components=n_components_optimal)
X_pca_elbow = pca_elbow.fit_transform(X_scaled)

# Lưu kết quả PCA từ Elbow Method vào cột mới
df['pcaelbow_to_ann'] = list(X_pca_elbow)

# Lưu cấu trúc PCA từ Elbow Method để áp dụng cho tập val và test
with open('pca_elbow.pkl', 'wb') as f:
    pickle.dump(pca_elbow, f)

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# Sử dụng PLSRegression để giảm chiều về 2
pls = PLSRegression(n_components=2)
X_pls = pls.fit_transform(X_scaled, df['label'])[0]

# Lưu kết quả PLS vào DataFrame
df['pls_to_ann'] = list(X_pls)

# Trực quan hóa dữ liệu sau khi giảm chiều
# Định nghĩa colormap cho 3 nhãn
cmap = ListedColormap(['r', 'g', 'b'])  # Đỏ cho nhãn 0, xanh lá cho nhãn 1, xanh dương cho nhãn 2

# Trực quan hóa dữ liệu sau khi giảm chiều
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pls[:, 0], X_pls[:, 1], c=df['label'], cmap=cmap, edgecolor='k')

# Gắn nhãn cho trục và tiêu đề
plt.xlabel('PLS Component 1')
plt.ylabel('PLS Component 2')
plt.title('PLS Reduced Data Visualization')

# Tạo colorbar với các nhãn
cbar = plt.colorbar(scatter, ticks=[0, 1, 2])
cbar.set_label('Label')
cbar.set_ticks([0, 1, 2])  # Thiết lập chính xác 3 giá trị nhãn
cbar.set_ticklabels(['Class 0', 'Class 1', 'Class 2'])

# Hiển thị biểu đồ
plt.show()

In [ ]:
df.drop(columns=['path_to_image', 'predicted_vector', 'predicted_label'], inplace=True)
df.head()

# Sau khi có thêm PLS, lưu lại DataFrame thành CSV
df.to_csv('train_set_processed_data_with_pca_pls.csv', index=False)

# Preprocessing data for val & test set

In [ ]:
df_val = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/val_not_aug.csv')
df_test = pd.read_csv('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/test_not_aug.csv')

...

In [ ]:
# Load lại PCA đã lưu
with open('pca_95.pkl', 'rb') as f:
    pca_95_loaded = pickle.load(f)

with open('pca_elbow.pkl', 'rb') as f:
    pca_elbow_loaded = pickle.load(f)

## More work to do here

In [ ]:
# import pandas as pd
# from sklearn.decomposition import PCA
# from sklearn.cross_decomposition import PLSRegression
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import joblib
# import os
# import numpy as np

# # Đọc dữ liệu từ train set
# df_train = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/train_not_aug.csv")
# df_val = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/val_not_aug.csv")
# df_test = pd.read_csv("/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/data/temp/test_not_aug.csv")

# # Chuyển đổi mỗi chuỗi thành mảng numpy
# df_train['predicted_vector'] = df_train['predicted_vector'].apply(lambda x: np.array(eval(x)))
# df_val['predicted_vector'] = df_val['predicted_vector'].apply(lambda x: np.array(eval(x)))
# df_test['predicted_vector'] = df_test['predicted_vector'].apply(lambda x: np.array(eval(x)))

# # Scale dữ liệu trước khi áp dụng PCA/PLS
# X_train_scaled = df_train['predicted_vector']
# X_val_scaled = df_val['predicted_vector']
# X_test_scaled = df_test['predicted_vector']

# # PCA giữ 95% thông tin
# pca_95 = PCA(0.99)  # n_components=0.95
# X_train_pca95 = pca_95.fit_transform(X_train_scaled)
# X_val_pca95 = pca_95.transform(X_val_scaled)
# X_test_pca95 = pca_95.transform(X_test_scaled)

# # Lưu PCA 95 để sử dụng sau này
# joblib.dump(pca_95, './pca_95.pkl')

# # Lưu lại PCA này và thêm vào df_train, df_val, df_test
# df_train['pca95_to_ann'] = X_train_pca95.tolist()
# df_val['pca95_to_ann'] = X_val_pca95.tolist()
# df_test['pca95_to_ann'] = X_test_pca95.tolist()

# # PCA bằng elbow method
# pca = PCA()
# pca.fit(X_train_scaled)
# # Tính variance explained ratio
# explained_variance = pca.explained_variance_ratio_
# # Elbow Method
# plt.plot(range(1, len(explained_variance)+1), np.cumsum(explained_variance), marker='o', linestyle='--')
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.title('PCA Elbow Method')
# plt.show()

In [ ]:
# pca_elbow = PCA(n_components=5)  # Chọn số chiều từ elbow method đã xác định trước
# X_train_pca_elbow = pca_elbow.fit_transform(X_train_scaled)
# X_val_pca_elbow = pca_elbow.transform(X_val_scaled)
# X_test_pca_elbow = pca_elbow.transform(X_test_scaled)

# # Lưu PCA elbow để sử dụng sau này
# joblib.dump(pca_elbow, './pca_elbow.pkl')

# # Lưu lại PCA này và thêm vào df_train, df_val, df_test
# df_train['pcaelbow_to_ann'] = X_train_pca_elbow.tolist()
# df_val['pcaelbow_to_ann'] = X_val_pca_elbow.tolist()
# df_test['pcaelbow_to_ann'] = X_test_pca_elbow.tolist()

# # PLS Regression để giảm về 2 chiều
# pls = PLSRegression(n_components=2)
# X_train_pls = pls.fit_transform(X_train_scaled, df_train['label'])[0]
# X_val_pls = pls.transform(X_val_scaled)
# X_test_pls = pls.transform(X_test_scaled)

# # Lưu PLS model để sử dụng sau này
# joblib.dump(pls, './pls_model.pkl')

# # Trực quan hóa tập train sau khi PLS
# plt.figure(figsize=(8, 6))
# plt.scatter(X_train_pls[:, 0], X_train_pls[:, 1], c=df_train['label'], cmap='viridis', edgecolor='k')
# plt.xlabel('PLS Component 1')
# plt.ylabel('PLS Component 2')
# plt.title('PLS Reduced Data Visualization (Train Set)')
# # Thêm thanh màu (colorbar) và đảm bảo rằng chỉ có 3 nhãn (0, 1, 2)
# cbar = plt.colorbar(scatter, ticks=[0, 1, 2])
# cbar.set_label('Label')
# plt.show()

# # Lưu lại kết quả PLS vào dataframe
# df_train['pls_to_ann'] = X_train_pls.tolist()
# df_val['pls_to_ann'] = X_val_pls.tolist()
# df_test['pls_to_ann'] = X_test_pls.tolist()

# # Lưu file CSV cho train, val và test
# df_train.to_csv("./train_set_with_pca_pls.csv", index=False)
# df_val.to_csv("./val_set_with_pca_pls.csv", index=False)
# df_test.to_csv("./test_set_with_pca_pls.csv", index=False)

# # Load lại dữ liệu để chuẩn bị cho deep learning
# df_train = pd.read_csv("./train_set_with_pca_pls.csv")
# df_val = pd.read_csv("./val_set_with_pca_pls.csv")
# df_test = pd.read_csv("./test_set_with_pca_pls.csv")

# # Lựa chọn các trường để train mạng deep learning
# X_train = df_train['pca95_to_ann'].apply(eval).tolist()
# y_train = df_train['label'].values

# X_val = df_val['pca95_to_ann'].apply(eval).tolist()
# y_val = df_val['label'].values

# X_test = df_test['pca95_to_ann'].apply(eval).tolist()
# y_test = df_test['label'].values

# # Bây giờ bạn có thể sử dụng X_train, y_train, X_val, y_val, X_test, y_test để train mô hình deep learning

# # Load lại mô hình PCA/PLS đã lưu để sử dụng tiếp
# scaler = joblib.load('./scaler.pkl')
# pca_95 = joblib.load('./pca_95.pkl')
# pca_elbow = joblib.load('./pca_elbow.pkl')
# pls = joblib.load('./pls_model.pkl')

In [3]:
import pandas as pd
import ast
import numpy as np
import json

def process_data(train_file, val_file, test_file):
    # Hàm chuyển predicted_vector từ string thành list
    def to_ann_39(vec_str):
        return np.array(ast.literal_eval(vec_str)).flatten()
    
    def to_transformer(vec_str):
        return np.array(ast.literal_eval(vec_str))

    # Đọc và xử lý các tập dữ liệu
    datasets = {'train': train_file, 'val': val_file, 'test': test_file}
    for key, file in datasets.items():
        df = pd.read_csv(file)
        df['to_ann_39'] = df['predicted_vector'].apply(to_ann_39)
        df['to_transformer'] = df['predicted_vector'].apply(to_transformer)

        # Lưu các mảng NumPy dưới dạng JSON
        df['to_ann_39'] = df['to_ann_39'].apply(lambda x: json.dumps(x.tolist()))  # Lưu dưới dạng JSON
        df['to_transformer'] = df['to_transformer'].apply(lambda x: json.dumps(x.tolist()))  # Lưu dưới dạng JSON

        # Lưu lại các file csv với 2 cột label và feature tương ứng
        df[['label', 'to_ann_39']].to_csv(f'{key}_to_ann_39.csv', index=False)
        df[['label', 'to_transformer']].to_csv(f'{key}_to_transformer.csv', index=False)


# Gọi hàm với các file CSV tương ứng
import os
os.chdir('../data/temp')
# process_data('train_not_aug.csv', 'val_not_aug.csv', 'test_not_aug.csv')

In [4]:
import pandas as pd
import torch
import json

def load_data_to_ann39_for_pytorch(file_path):
    # Đọc dữ liệu
    df = pd.read_csv(file_path)

    # Chuyển đổi các chuỗi JSON thành numpy array
    df['to_ann_39'] = df['to_ann_39'].apply(lambda x: np.array(json.loads(x)))

    # Chuyển đổi sang PyTorch tensor
    features = torch.tensor(np.vstack(df['to_ann_39'].values)).float()
    labels = torch.tensor(df['label'].values).long()

    return features, labels

def load_data_to_transformer_for_pytorch(file_path):
    # Đọc dữ liệu
    df = pd.read_csv(file_path)

    # Chuyển đổi các chuỗi JSON thành numpy array
    df['to_transformer'] = df['to_transformer'].apply(lambda x: np.array(json.loads(x)))

    # Chuyển đổi sang PyTorch tensor
    features = torch.tensor(np.vstack(df['to_transformer'].values)).float()
    labels = torch.tensor(df['label'].values).long()

    return features, labels

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Định nghĩa mạng ANN
class ANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        # self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        # x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Hàm tính trọng số của các label cho loss function
def compute_class_weights(labels):
    unique_labels, counts = torch.unique(labels, return_counts=True)
    class_weights = len(labels) / counts.float()
    weight_dict = {label.item(): class_weights[i].item() for i, label in enumerate(unique_labels)}
    weights = torch.tensor([weight_dict[label.item()] for label in unique_labels])
    return weights

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Hàm tính toán accuracy và F1 score
def calculate_metrics(labels, outputs):
    _, preds = torch.max(outputs, 1)
    acc = torch.sum(preds == labels).item() / labels.size(0)
    f1 = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
    return acc, f1

# Cải tiến hàm train model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, early_stop_patience=10):
    model.to(device)
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_train_labels = []
        all_train_outputs = []

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_train_labels.append(labels)
            all_train_outputs.append(outputs)

        # Tính toán train accuracy và F1 score
        all_train_labels = torch.cat(all_train_labels)
        all_train_outputs = torch.cat(all_train_outputs)
        train_acc, train_f1 = calculate_metrics(all_train_labels, all_train_outputs)

        # Evaluate trên tập validation
        val_loss, val_acc, val_f1 = evaluate_model(model, val_loader, criterion, device)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, "
              f"Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # Reset patience counter if validation loss decreases
        else:
            patience_counter += 1

        if patience_counter >= early_stop_patience:
            print(f"Stopping early at epoch {epoch+1}")
            break

# Hàm đánh giá mô hình với thêm val F1
def evaluate_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    all_labels = []
    all_outputs = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.append(labels)
            all_outputs.append(outputs)

    all_labels = torch.cat(all_labels)
    all_outputs = torch.cat(all_outputs)

    accuracy = correct / total
    f1 = f1_score(all_labels.cpu().numpy(), torch.argmax(all_outputs, dim=1).cpu().numpy(), average='weighted')
    avg_loss = total_loss / len(data_loader)
    
    return avg_loss, accuracy, f1

# Hàm test với thêm confusion matrix
def test_model(model, test_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            all_labels.append(labels)
            all_predictions.append(predicted)

    all_labels = torch.cat(all_labels)
    all_predictions = torch.cat(all_predictions)

    accuracy = torch.sum(all_predictions == all_labels).item() / len(all_labels)
    f1 = f1_score(all_labels.cpu().numpy(), all_predictions.cpu().numpy(), average='weighted')

    # Ma trận nhầm lẫn (confusion matrix)
    cm = confusion_matrix(all_labels.cpu().numpy(), all_predictions.cpu().numpy())
    
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    return accuracy, f1, cm

In [18]:
# Load data cho các tập train, val, test (giả sử các hàm load_data_to_ann39_for_pytorch đã tồn tại)
train_features, train_labels = load_data_to_ann39_for_pytorch('train_to_ann_39.csv')
val_features, val_labels = load_data_to_ann39_for_pytorch('val_to_ann_39.csv')
test_features, test_labels = load_data_to_ann39_for_pytorch('test_to_ann_39.csv')

# Tạo TensorDataset và DataLoader cho tập train, val, test
batch_size = 64
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
# Khởi tạo model
input_dim = train_features.shape[1]
hidden_dim = 97
output_dim = len(torch.unique(train_labels))  # Số lớp output tương ứng với số nhãn
dropout_prob = 0.9

model = ANN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, dropout_prob=dropout_prob)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tính trọng số cho loss function
class_weights = compute_class_weights(train_labels)
class_weights = class_weights.to(device)  # Đảm bảo trọng số được chuyển lên GPU nếu có
print(f'class_weights={class_weights}\nShape of class_weights={class_weights.shape}')

# Loss function và optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)  # Dùng CrossEntropyLoss với trọng số của nhãn
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện model
num_epochs = 100
device = torch.device(device)

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

# Đánh giá trên tập test
test_model(model, test_loader, device)


class_weights=tensor([3.5751, 3.4293, 2.3327])
Shape of class_weights=torch.Size([3])
Epoch [1/100], Loss: 32.4340, Train Acc: 0.5206, Train F1: 0.5052, Val Loss: 0.2301, Val Acc: 0.9259, Val F1: 0.9263
Epoch [2/100], Loss: 7.9401, Train Acc: 0.8344, Train F1: 0.8334, Val Loss: 0.2005, Val Acc: 0.9444, Val F1: 0.9447
Epoch [3/100], Loss: 4.3801, Train Acc: 0.9097, Train F1: 0.9109, Val Loss: 0.2061, Val Acc: 0.9444, Val F1: 0.9447
Epoch [4/100], Loss: 3.7682, Train Acc: 0.9184, Train F1: 0.9171, Val Loss: 0.2155, Val Acc: 0.9407, Val F1: 0.9411
Epoch [5/100], Loss: 3.2146, Train Acc: 0.9247, Train F1: 0.9257, Val Loss: 0.2239, Val Acc: 0.9407, Val F1: 0.9411
Epoch [6/100], Loss: 2.3462, Train Acc: 0.9525, Train F1: 0.9534, Val Loss: 0.2327, Val Acc: 0.9407, Val F1: 0.9411
Epoch [7/100], Loss: 2.6675, Train Acc: 0.9445, Train F1: 0.9452, Val Loss: 0.2388, Val Acc: 0.9519, Val F1: 0.9520
Epoch [8/100], Loss: 2.6650, Train Acc: 0.9398, Train F1: 0.9402, Val Loss: 0.2397, Val Acc: 0.9407, 

(0.8860294117647058,
 0.8866754543482271,
 array([[ 57,   0,   4],
        [  0,  81,  15],
        [  0,  12, 103]]))

In [5]:
# Ví dụ load file train_to_transformer.csv
train_features, train_labels = load_data_to_transformer_for_pytorch('train_to_transformer.csv')
print(train_features.shape, train_labels.shape)

# Ví dụ load file val_to_transformer.csv
val_features, val_labels = load_data_to_transformer_for_pytorch('val_to_transformer.csv')
print(val_features.shape, val_labels.shape)

# Ví dụ load file test_to_transformer.csv
test_features, test_labels = load_data_to_transformer_for_pytorch('test_to_transformer.csv')
print(test_features.shape, test_labels.shape)

torch.Size([16406, 3]) torch.Size([1262])
torch.Size([3510, 3]) torch.Size([270])
torch.Size([3536, 3]) torch.Size([272])


In [10]:
train_features = train_features.view(-1, 13, 3)
val_features = val_features.view(-1, 13, 3)
test_features = test_features.view(-1, 13, 3)

print(train_features.shape, val_features.shape, test_features.shape)

torch.Size([1262, 13, 3]) torch.Size([270, 13, 3]) torch.Size([272, 13, 3])


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Tạo TensorDataset và DataLoader cho tập train, val, test
batch_size = 64
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [23]:
num_patches = 13
num_classes = 3
dim = 9

a = torch.zeros(1, num_patches, num_classes, dim)
print(a.shape)

# shape of 

torch.Size([1, 13, 3, 9])


In [1]:
import sys
sys.path.append('/mnt/DataSamsung/project/Research_ThyroidFNA_ClassAI/phase3_140924/src')
from models.module2.transformer import main, get_transformer_model

main()

Output: tensor([[ 0.6676,  0.2531, -0.3114],
        [ 0.3443, -0.0797, -0.2200]], grad_fn=<AddmmBackward0>)
Output shape: torch.Size([2, 3])
Total parameters: 3535


In [18]:
model = get_transformer_model()

device = 'cpu'

# Tính trọng số cho loss function
class_weights = compute_class_weights(train_labels)
class_weights = class_weights.to(device)  # Đảm bảo trọng số được chuyển lên GPU nếu có
print(f'class_weights={class_weights}\nShape of class_weights={class_weights.shape}')

# Loss function và optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)  # Dùng CrossEntropyLoss với trọng số của nhãn
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện model
num_epochs = 100

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

# Đánh giá trên tập test
test_model(model, test_loader, device)

class_weights=tensor([3.5751, 3.4293, 2.3327])
Shape of class_weights=torch.Size([3])
Epoch [1/100], Loss: 16.0506, Train Acc: 0.7710, Train F1: 0.7523, Val Loss: 0.7061, Val Acc: 0.9370, Val F1: 0.9370
Epoch [2/100], Loss: 12.6495, Train Acc: 0.8914, Train F1: 0.8857, Val Loss: 0.5798, Val Acc: 0.9519, Val F1: 0.9522
Epoch [3/100], Loss: 10.6413, Train Acc: 0.9279, Train F1: 0.9245, Val Loss: 0.4867, Val Acc: 0.9481, Val F1: 0.9484
Epoch [4/100], Loss: 8.7148, Train Acc: 0.9263, Train F1: 0.9222, Val Loss: 0.4060, Val Acc: 0.9519, Val F1: 0.9521
Epoch [5/100], Loss: 7.3521, Train Acc: 0.9263, Train F1: 0.9229, Val Loss: 0.3450, Val Acc: 0.9481, Val F1: 0.9485
Epoch [6/100], Loss: 6.0393, Train Acc: 0.9445, Train F1: 0.9418, Val Loss: 0.3031, Val Acc: 0.9519, Val F1: 0.9522
Epoch [7/100], Loss: 4.9801, Train Acc: 0.9540, Train F1: 0.9523, Val Loss: 0.2752, Val Acc: 0.9519, Val F1: 0.9522
Epoch [8/100], Loss: 4.5536, Train Acc: 0.9564, Train F1: 0.9545, Val Loss: 0.2992, Val Acc: 0.9370

(0.8970588235294118,
 0.8976833719913304,
 array([[ 58,   2,   1],
        [  0,  84,  12],
        [  0,  13, 102]]))